# Recommendation System
### Using Matrix Factorization Embeddings and Deep Feed Forward Regressor

The Embeddings and Regressor will be trained using MovieLens 20M Dataset.

References:
F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4, Article 19 (December 2015), 19 pages. DOI=<http://dx.doi.org/10.1145/2827872>

Exploratory Data Analysis, Model Definition, Training and Fine-Tunning described step by step on this Notebook

In [0]:
from IPython.display import clear_output
!pip install --upgrade tensorflow-gpu
clear_output()

In [0]:
#Downloads and extract Dataset to local, wait for download, i dont want to put a progress bar here sorry
#You can run this on google colab for get faster downloads speeds
import os
import zipfile
import requests

if(not os.path.exists("./Datasets/MoviLens20M.zip")):

  resp = requests.get("http://files.grouplens.org/datasets/movielens/ml-20m.zip")

  os.mkdir("./Datasets")

  with open("./Datasets/MoviLens20M.zip", "wb") as f:
    f.write(resp.content)

  with zipfile.ZipFile("./Datasets/MoviLens20M.zip", "r") as zip_ref:
    zip_ref.extractall("./Datasets")


In [0]:
#Imports
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Exploratory Data Analysis

Simple exploration without too much fancy graphs

In [52]:
#Loads Dataset, we only need ratings.csv and movies.csv files, we can drop timestamp and genres for now
ratings_df = pd.read_csv("./Datasets/ml-20m/ratings.csv").drop(["timestamp"], axis=1)
movies_df = pd.read_csv("./Datasets/ml-20m/movies.csv").drop(["genres"], axis=1)

ml_df = ratings_df.merge(movies_df, on="movieId")

ml_df = ml_df.reindex(columns=["userId", "movieId", "title", "rating"])
ml_df.head()

,userId,movieId,title,rating
0,1,2,Jumanji (1995),3.5
1,5,2,Jumanji (1995),3.0
2,13,2,Jumanji (1995),3.0
3,29,2,Jumanji (1995),3.0
4,34,2,Jumanji (1995),3.0


In [53]:
#Check info about the Dataset
ml_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000263 entries, 0 to 20000262
Data columns (total 4 columns):
userId     int64
movieId    int64
title      object
rating     float64
dtypes: float64(1), int64(2), object(1)
memory usage: 762.9+ MB


In [54]:
#Check for NaNs
ml_df.isna().sum()

userId     0
movieId    0
title      0
rating     0
dtype: int64

In [55]:
#List unique values of each column
n_users = ml_df["userId"].max()
n_movies = ml_df["movieId"].nunique()

print("Unique Users: " + str(n_users))
print("Unique Movies: " + str(n_movies))

Unique Users: 138493
Unique Movies: 26744


In [56]:
#Top movies with more rating count (dont confuse with more views or more rating score, but are correlated)
count = ml_df["title"].value_counts()
count[:15]

Pulp Fiction (1994)                          67310
Forrest Gump (1994)                          66172
Shawshank Redemption, The (1994)             63366
Silence of the Lambs, The (1991)             63299
Jurassic Park (1993)                         59715
Star Wars: Episode IV - A New Hope (1977)    54502
Braveheart (1995)                            53769
Terminator 2: Judgment Day (1991)            52244
Matrix, The (1999)                           51334
Schindler's List (1993)                      50054
Toy Story (1995)                             49695
Fugitive, The (1993)                         49581
Apollo 13 (1995)                             47777
Independence Day (a.k.a. ID4) (1996)         47048
Usual Suspects, The (1995)                   47006
Name: title, dtype: int64

## Preprocessing

In [57]:
#Normalize ratings
ml_df["rating_norm"] = ml_df["rating"] / 5.0
ml_df["userId"] = ml_df["userId"].astype("category").cat.codes.values
ml_df["movieId"] = ml_df["movieId"].astype("category").cat.codes.values
ml_df.head()

,userId,movieId,title,rating,rating_norm
0,0,1,Jumanji (1995),3.5,0.7
1,4,1,Jumanji (1995),3.0,0.6
2,12,1,Jumanji (1995),3.0,0.6
3,28,1,Jumanji (1995),3.0,0.6
4,33,1,Jumanji (1995),3.0,0.6


In [0]:
users = ml_df["userId"].values
movies = ml_df["movieId"].values
ratings = ml_df["rating_norm"].values.reshape([-1, 1])

In [0]:
#Create Datasets for train, evaluation and testing, and a full version of the dataset
ml_ds = tf.data.Dataset.from_tensor_slices(({"userId":users, "movieId":movies}, ratings)).shuffle(2048)
full_ds = ml_ds
eval_ds = ml_ds.take(10000).batch(10000)
ml_ds = ml_ds.skip(10000)
test_ds = ml_ds.take(500000).batch(50000)
train_ds = ml_ds.skip(500000)

## Matrix Factorization

Optimize embeddings for users and movies. This embeddings will be used later in the Regressor. For train the Factorizer, full dataset will be used, for optimal embeddings aver all the users / movies

In [0]:
#Model Definition
class MatrixFactorizer(tf.keras.Model):
  """This model will be used for optimize the embeddings, later will be discarded, just keeping the embedding layers weights"""

  def __init__(self, users, movies, emb_dim):
    super(MatrixFactorizer, self).__init__()

    self.user_emb = tf.keras.layers.Embedding(users, emb_dim)
    self.user_flat = tf.keras.layers.Flatten()

    self.movie_emb = tf.keras.layers.Embedding(movies, emb_dim)
    self.movie_flat = tf.keras.layers.Flatten()

    self.dot = tf.keras.layers.Dot(axes=1)

  def call(self, inputs):
    X_user = self.user_emb(inputs["userId"])
    X_user = self.user_flat(X_user)

    X_movie = self.movie_emb(inputs["movieId"])
    X_movie = self.movie_flat(X_movie)

    X = self.dot([X_user, X_movie])
    return X

In [0]:
#Matrix Factorizer Hyperparams
emb_dim = 4
lr = 0.001
epochs = 15
batch_size = 40960

In [62]:
#Model instantiation
factorizer = MatrixFactorizer(n_users, n_movies, emb_dim)
factorizer.compile(tf.keras.optimizers.Adam(lr), tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.RootMeanSquaredError()])
factorizer.train_on_batch(train_ds.batch(1))
factorizer.summary()

Model: "matrix_factorizer_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     multiple                  553972    
_________________________________________________________________
flatten_14 (Flatten)         multiple                  0         
_________________________________________________________________
embedding_15 (Embedding)     multiple                  106976    
_________________________________________________________________
flatten_15 (Flatten)         multiple                  0         
_________________________________________________________________
dot_7 (Dot)                  multiple                  0         
Total params: 660,948
Trainable params: 660,948
Non-trainable params: 0
_________________________________________________________________


In [63]:
#Model fitting
factorizer.fit(full_ds.batch(batch_size), epochs=epochs)

Epoch 1/15
489/489 [==============================] - 73s 148ms/step - loss: 0.5413 - root_mean_squared_error: 0.7358
Epoch 2/15
489/489 [==============================] - 71s 146ms/step - loss: 0.5387 - root_mean_squared_error: 0.7339
Epoch 3/15
489/489 [==============================] - 73s 148ms/step - loss: 0.5123 - root_mean_squared_error: 0.7157
Epoch 4/15
489/489 [==============================] - 71s 146ms/step - loss: 0.4266 - root_mean_squared_error: 0.6531
Epoch 5/15
489/489 [==============================] - 72s 147ms/step - loss: 0.2949 - root_mean_squared_error: 0.5430
Epoch 6/15
489/489 [==============================] - 71s 145ms/step - loss: 0.1742 - root_mean_squared_error: 0.4173
Epoch 7/15
489/489 [==============================] - 72s 147ms/step - loss: 0.1013 - root_mean_squared_error: 0.3182
Epoch 8/15
489/489 [==============================] - 72s 147ms/step - loss: 0.0661 - root_mean_squared_error: 0.2571
Epoch 9/15
489/489 [==============================] - 72

In [64]:
factorizer.evaluate(test_ds)

10/10 [==============================] - 2s 165ms/step - loss: 0.0292 - root_mean_squared_error: 0.1710


[0.02924454864114523, 0.17101038]